In [55]:
import pandas as pd

In [56]:
intensity = pd.read_csv('intensity.csv') # intensity	year
intensity

,intensity,year
0,0.517203,2011
1,0.516045,2012
2,0.516379,2013
3,0.514491,2014
4,0.510113,2015
5,0.507194,2016
6,0.505843,2017
7,0.504636,2018
8,0.515194,2019
9,0.515194,2020


In [57]:
cambridge = pd.read_csv('orig/2_cambridge.csv') # Date and Time,"Hydro-only, MtCO2e","Estimated, MtCO2e","Coal-only, MtCO2e"
cambridge['Date and Time'] = pd.to_datetime(cambridge['Date and Time'])
cambridge.index = cambridge['Date and Time']
cambridge = cambridge[["Estimated, MtCO2e"]]
#yearly avg
cambridge = cambridge.resample('Y').mean()
cambridge["year"] = cambridge.index.year
cambridge.rename(columns={"Estimated, MtCO2e": "cambridge"}, inplace=True)
cambridge["cambridge"] = cambridge["cambridge"] * 1e6
cambridge.to_csv('2_cambridge.csv',index=False)
cambridge

,cambridge,year
Date and Time,,
2010-12-31,1.453766e+03,2010
2011-12-31,7.755366e+04,2011
2012-12-31,5.489394e+04,2012
2013-12-31,5.920221e+05,2013
2014-12-31,2.626312e+06,2014
2015-12-31,1.962968e+06,2015
2016-12-31,3.052266e+06,2016
2017-12-31,6.855055e+06,2017
2018-12-31,2.282154e+07,2018


In [58]:
digiconomist = pd.read_csv('orig/3_digiconomist.csv') # "DateTime","Estimated TWh per Year","Minimum TWh per Year"
digiconomist['DateTime'] = pd.to_datetime(digiconomist['DateTime'])
digiconomist.index = digiconomist['DateTime']
digiconomist = digiconomist[["Estimated TWh per Year"]]

#yearly avg
digiconomist = digiconomist.resample('Y').mean()
digiconomist["year"] = digiconomist.index.year
digiconomist.rename(columns={"Estimated TWh per Year": "digiconomist"}, inplace=True)

#convert to co2
digiconomist = digiconomist.merge(intensity, on="year", how="left")
digiconomist["digiconomist"] = digiconomist["digiconomist"] * digiconomist["intensity"] * 1e6
digiconomist.drop(columns=["intensity"], inplace=True)

digiconomist.to_csv('3_digiconomist.csv',index=False)

In [59]:
coinmetrics = pd.read_csv('orig/4_coinmetrics.csv') # year,coinmetrics
#convert gw to Twh per year
coinmetrics["coinmetrics"] = coinmetrics["coinmetrics"] * 8.76
#convert to co2
coinmetrics = coinmetrics.merge(intensity, on="year", how="left")
coinmetrics["coinmetrics"] = coinmetrics["coinmetrics"] * coinmetrics["intensity"] * 1e6
coinmetrics.drop(columns=["intensity"], inplace=True)

coinmetrics.to_csv('4_coinmetrics.csv',index=False)

In [ ]:
rest = pd.read_csv('table/table.csv') # Author,Publication date,Title,Estimate (TWh)

# for each row, create a csv with the author name
for index, row in rest.iterrows():
    name = row["Author"].split(",")[0]
    twh = row["Estimate (TWh)"]
    year = row["Publication date"].split(" ")[-1]

    #convert to co2
    intensity_year = intensity[intensity["year"] == int(year)]["intensity"].values[0]
    co2 = twh * intensity_year * 1e6

    with open(f'orig/{index+10}_{name}.csv', 'w') as f:
        f.write(f'year,{name}\n')
        f.write(f'{year},{co2}\n')